## 3. 텍스트를 분석가능한 형태로 바꾸는 다양한 방법 테스트

### 3.1. One-hot encoding, bag-of-words

In [ ]:
# nltk, konlpy 패키지 설치
!pip install nltk
!pip install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 9.0 MB/s 
     |████████████████████████████████| 453 kB 33.6 MB/s 


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# 문장 분리를 위한 패키지
from nltk import sent_tokenize
# 형태소 분석을 위한 패키지
from konlpy.tag import Kkma
# 데이터 프레임 사용을 위한 패키지
import pandas as pd

In [ ]:
# bag-of-words 생성을 위한 패키지 로드
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
corpus = [
    "KOSPI 지수가 하락했다.",
    "원달러 환율이 하락했다.",
    "KOSPI 지수가 연이어 상승하고 있다.",
    "대출금리가 하락했다."
]

vectorizer = CountVectorizer(min_df = 1, ngram_range = (1,1))
features = vectorizer.fit_transform(corpus)
vocab = vectorizer.get_feature_names()

bows = pd.DataFrame(features.toarray(), columns=vocab)
bows.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,kospi,대출금리가,상승하고,연이어,원달러,있다,지수가,하락했다,환율이
0,1,0,0,0,0,0,1,1,0
1,0,0,0,0,1,0,0,1,1
2,1,0,1,1,0,1,1,0,0
3,0,1,0,0,0,0,0,1,0


### 3.2. 빈도행렬 (TDM : Term-Document Matrix)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# 문장 목록
corpus = [
    'KOSPI 지수가 하락했다 하락했다',
    '전일 급등한 KOSPI 지수는 금일 하락했다',
    '원달러 환율이 요동치고 있다',
    '환율이 급등함에 따라 KOSPI 지수가 하락했다'
]

vector = CountVectorizer()

# 문장 목록으로 부터 각 단어의 빈도수를 기록
print(vector.fit_transform(corpus).toarray())

# 각 단어와 맵핑된 인덱스 출력
print(vector.vocabulary_)

[[1 0 0 0 0 0 0 0 0 1 0 2 0]
 [1 1 1 0 0 0 0 0 1 0 1 1 0]
 [0 0 0 0 0 1 1 1 0 0 0 0 1]
 [1 0 0 1 1 0 0 0 0 1 0 1 1]]
{'kospi': 0, '지수가': 9, '하락했다': 11, '전일': 8, '급등한': 2, '지수는': 10, '금일': 1, '원달러': 6, '환율이': 12, '요동치고': 5, '있다': 7, '급등함에': 3, '따라': 4}


### 3.3. 동반출현행렬(Co-occurence Matrix)

In [ ]:
import numpy as np
from konlpy.tag import Kkma

In [ ]:
# 문장 목록
corpus = [
  'KOSPI 지수가 하락했다',
  '전일 급등한 KOSPI 지수는 금일 하락했다',
  '원달러 환율이 요동치고 있다',
  '환율이 급등함에 따라 KOSPI 지수가 하락했다'
]

# 형태소 분석기
kkma = Kkma()
# 명사만 추출한 결과 저장
nouns_list = []
# 단어 사전
dictionary = {}
# 단어 인덱스
i = 0

In [ ]:
# 단어사전 생성
for c in corpus:
  # 문장에서 명사만 추출
  nouns = kkma.nouns(c)
  nouns_list.append(nouns)
  for n in nouns:
    # 사전에 없는 경우 추가하고 인덱스 부여
    if n not in dictionary.keys():
      dictionary[n] = i
      i += 1    

# 동반출현행렬 생성
co_mat = np.zeros((len(dictionary.keys()), len(dictionary.keys()))).astype(int)
for n in nouns_list:
  for i1 in range(len(n)):    
    w1_idx = dictionary[n[i1]]
    co_mat[w1_idx, w1_idx] = 1
    for i2 in range(i1+1, len(n)):      
      w2_idx = dictionary[n[i2]]
      co_mat[w1_idx, w2_idx] += 1
      co_mat[w2_idx, w1_idx] += 1

In [ ]:
print("명사 인덱스 :", dictionary)
print("\n동반출현행렬")
print(co_mat)

명사 인덱스 : {'지수': 0, '하락': 1, '전일': 2, '급등': 3, '금일': 4, '원': 5, '원달러': 6, '달러': 7, '환율': 8}

동반출현행렬
[[1 3 1 2 1 0 0 0 1]
 [3 1 1 2 1 0 0 0 1]
 [1 1 1 1 1 0 0 0 0]
 [2 2 1 1 1 0 0 0 1]
 [1 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 1 1 1 1]
 [0 0 0 0 0 1 1 1 1]
 [1 1 0 1 0 1 1 1 1]]


### 3.4. Word2Vec

In [ ]:
# 테스트에 사용할 데이터를 다운로드 (뉴스기사)
!wget https://raw.githubusercontent.com/initialsw/TextDataLecture/main/연수용%20뉴스기사.csv

--2022-10-03 07:02:10--  https://raw.githubusercontent.com/initialsw/TextDataLecture/main/%EC%97%B0%EC%88%98%EC%9A%A9%20%EB%89%B4%EC%8A%A4%EA%B8%B0%EC%82%AC.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3486073 (3.3M) [text/plain]
Saving to: ‘연수용 뉴스기사.csv’

연수용 뉴스기사.csv 100%[===================>]   3.32M  --.-KB/s    in 0.03s   

2022-10-03 07:02:10 (98.0 MB/s) - ‘연수용 뉴스기사.csv’ saved [3486073/3486073]



In [ ]:
# 데이터 프레임 로드를 위한 패키지
import pandas as pd

data = pd.read_csv("연수용 뉴스기사.csv", encoding='utf-8', sep='\t')
# 앞의 50개 기사만 사용
data = data.iloc[:50]
data.head()

,날짜,제목,언론사명,본문,URL
0,20220901,"""주식 팔고 대피""…두달새 46조, 은행 예·적금에 '뭉칫돈'",머니투데이,김영운 기자 = 지난 2분기 가계 빚이 사상 최대를 기록한 가운데 24일 오전 경기...,https://n.news.naver.com/mnews/article/008/000...
1,20220901,2분기 경제성장률 0.7%…수출 줄었지만 민간소비로 버텨,연합뉴스,"민간소비 2.9%, 속보치보다 0.1%p↓…수출 -3.1% 2분기 실질 국민총소득 ...",https://n.news.naver.com/mnews/article/001/001...
2,20220901,"전·현직 경제관료, 책임론 다시 불붙나",서울신문,"추경호·김주현 등 의사결정 참여한덕수는 론스타측 김앤장 고문 정부, 론스타 요구 6...",https://n.news.naver.com/mnews/article/081/000...
3,20220901,잇단 금리 인상에...서울 아파트값 3년 7개월 만에 최대 하락,ytn,금리 인상과 거래 침체 여파로 서울을 비롯한 수도권 아파트값 하락 폭이 확대됐습니다...,https://n.news.naver.com/mnews/article/052/000...
4,20220901,무역적자 100억弗 육박… 66년 만에 최대치,세계일보,14년 만에 5개월 연속 적자반도체·대중수출 뒷걸음 탓소비 힘입어 2분기 성장 0....,https://n.news.naver.com/mnews/article/022/000...


In [ ]:
from konlpy.tag import Kkma
from nltk import sent_tokenize

# 형태소 분석기 초기화
kkma = Kkma()
# 문장별로 명사를 추출한 결과
word_list = []

# 뉴스기사 텍스트에서 명사를 추출하여 Word2Vec 임베딩 실시
for article in data["본문"].tolist():
  # 문장 단위로 분리
  sentences = sent_tokenize(article)
  # 명사만 추출해서 리스트로 저장
  for s in sentences:        
    nouns = kkma.nouns(s)    
    if len(nouns) > 0:      
      new_nouns = []
      # 단어 길이가 1글자인것은 제외
      for n in nouns:
        if len(n) > 1:
          new_nouns.append(n)
      word_list.append(new_nouns)

In [ ]:
# 명사추출 결과 출력
for i in range(0, 10):
  print(word_list[i])

# 입력데이터 형식 -> word_list 로 저장

['김영운', '영운', '기자', '2분기', '분기', '가계', '사상', '최대', '기록', '가운데', '24', '24일', '오전', '경기', '수원', '수원시', '장안', '장안구', '도로', '도로변', '은행', '예금', '금리', '광고', '현수막']
['한국', '한국은행', '은행', '23', '23일', '2022', '2022년', '2분기', '분기', '가계', '가계신용', '신용', '기준', '잔액', '1869', '1869조4000억', '4000', '통계', '작성', '시작', '2003', '2003년', '이래', '규모']
['국내', '가계', '가계신용', '신용', '2013', '2013년', '2분기', '분기', '이후', '37', '37분기', '연속', '증가세']
['2022.8', '24', '뉴스', '뉴스1']
['무단', '전재', '배포', '금지', '5대', '은행', '정기', '적금', '80', '80조원', '육박', '뭉칫돈', '유입']
['주식', '가상', '가상자산', '자산', '위험', '위험자산', '안전', '안전처', '은행', '적금']
['통화', '긴축', '금리', '상승', '기조', '내년', '예상', '역머니무브', '머니', '무브', '현상', '전망']
['1일', '은행권', '국민', '신한', '하나', '우리', '농협', '농협은행', '은행', '5대', '8월', '정기', '적금', '잔액', '768', '768조5434억', '5434', '사이', '17', '17조9776억', '9776']
['7월', '달간', '28', '28조56억', '56', '증가', '46', '46조원', '시중', '시중자금', '자금', '은행', '계좌']
['한국', '한국은행', '은행', '물가', '상승', '압력', '완화', '7월', '빅스텝', '스텝', '지난달', '25', '25일', '기준',

In [ ]:
# Word2Vec 실행을 위한 gensim 패키지 설치
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from gensim.models import Word2Vec

model = Word2Vec(sentences = word_list, size = 16, window = 16, min_count = 1, workers = 8, sg = 1)

In [ ]:
print("벡터 크기 :", model.wv.vectors.shape)

벡터 크기 : (3948, 16)


In [ ]:
# 단어 "한국은행"의 좌표
print(model.wv.__getitem__("한국은행"))
print("\n")
# 단어 "물가"의 좌표
print(model.wv.__getitem__("물가"))

[ 0.09706859 -0.26565334 -0.26668411 -0.30333683  0.7122204  -0.13285439
  0.41959822 -0.02757252 -0.26712668 -0.26250577 -0.36558676  0.19404861
  0.05025736  0.6028545   1.0653424   0.6873924 ]


[ 0.12685238 -0.06184559 -0.39993638 -0.5269245   0.62292844 -0.10072646
  0.5346862   0.14724879 -0.1148377  -0.13616697 -0.3971643   0.22490874
 -0.01983843  0.5240946   0.9846654   0.28696644]


In [ ]:
# 단어간 유사도 측정
print(model.wv.similarity("물가", "금리"))
print(model.wv.similarity("금리", "대출"))

0.8627421
0.98764086


In [ ]:
# 가장 유사도가 높은 명사 20개 출력
model.wv.most_similar("한국은행", topn=20)

[('발표', 0.9869703054428101),
 ('2022', 0.9829413890838623),
 ('4분기', 0.9816467761993408),
 ('집계', 0.977138876914978),
 ('2022년', 0.9768857955932617),
 ('2.6', 0.9767464995384216),
 ('1분기', 0.9743006229400635),
 ('0.2', 0.9742050170898438),
 ('국민소득', 0.9741829633712769),
 ('계정', 0.9737166166305542),
 ('통계국', 0.9734715223312378),
 ('0.1', 0.9734529852867126),
 ('경제통계국', 0.9721899032592773),
 ('전망', 0.9714668989181519),
 ('대비', 0.9712716937065125),
 ('8864', 0.9701074957847595),
 ('1일', 0.9696728587150574),
 ('국내', 0.9680846929550171),
 ('중구', 0.9680601358413696),
 ('치인', 0.9671372175216675)]

In [ ]:
# 가장 유사도가 높은 명사 20개 출력
model.wv.most_similar("기준금리", topn=20)

[('기준', 0.9906155467033386),
 ('가계', 0.9869949817657471),
 ('금리', 0.9838302135467529),
 ('지난달', 0.9834046959877014),
 ('인상', 0.9784610271453857),
 ('적금', 0.9752289652824402),
 ('대출', 0.9752234220504761),
 ('전월', 0.9735909104347229),
 ('예금', 0.9725884199142456),
 ('가계대출', 0.9716573357582092),
 ('시장', 0.9703873991966248),
 ('7월', 0.9698838591575623),
 ('주택', 0.9683623313903809),
 ('잔액', 0.9677243232727051),
 ('5대', 0.9673600196838379),
 ('대출금리', 0.9664769172668457),
 ('기조', 0.965130090713501),
 ('국채', 0.964621901512146),
 ('채권', 0.9633598327636719),
 ('정기예금', 0.9625884294509888)]

In [ ]:
# 가장 유사도가 높은 명사 20개 출력
model.wv.most_similar("물가", topn=20)

[('행진', 0.9944055080413818),
 ('사상', 0.9939160346984863),
 ('6월', 0.9938833117485046),
 ('이환', 0.9928392171859741),
 ('급증', 0.992743968963623),
 ('185', 0.9922105073928833),
 ('기인', 0.992184042930603),
 ('긴축', 0.9921066761016846),
 ('고물가', 0.9920373558998108),
 ('요인', 0.9919002056121826),
 ('가운데', 0.9918802976608276),
 ('566', 0.9918131232261658),
 ('유가', 0.9918022155761719),
 ('수입단가', 0.9917970895767212),
 ('28.2', 0.9917669296264648),
 ('강세', 0.9916327595710754),
 ('누적', 0.991530179977417),
 ('우리나라', 0.9914841055870056),
 ('상승세', 0.9914799928665161),
 ('소비자물가', 0.9914271831512451)]